In [79]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 29 15:40:04 2025

@author: zemsk


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
import os


path = "./coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral_whole.csv")

df = pd.read_csv(filename_read)

# Removed only locations and labels, no real data touched
df.drop("Ocean_Name", axis=1, inplace=True)
df.drop("Country_Name", axis=1, inplace=True)
df.drop("Sample_ID", axis=1, inplace=True)
df.drop("Date_Year", axis=1, inplace=True)
df.drop("Bleaching_Level", axis=1, inplace=True)
df.drop("Realm_Name", axis=1, inplace=True)
# #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # For the sake of bigger dataset this feature is dropped
df.drop("Percent_Cover", axis=1, inplace=True)
df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
df.drop("Temperature_Maximum", inplace=True, axis=1)
#
label_encoder = LabelEncoder()
df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
#df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
# df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df.dropna(inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 32714 entries, 0 to 35044
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cyclone_Frequency  32714 non-null  float64
 1   Depth_m            32714 non-null  float64
 2   Distance_to_Shore  32714 non-null  float64
 3   Exposure           32714 non-null  int64  
 4   Turbidity          32714 non-null  float64
 5   SSTA               32714 non-null  float64
 6   TSA                32714 non-null  float64
 7   Percent_Bleaching  32714 non-null  float64
 8   Temperature_Mean   32714 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 2.5 MB


In [97]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

X = df.drop(columns=['Percent_Bleaching'])
y = df['Percent_Bleaching']

s_scaler = StandardScaler()
r_scaler = RobustScaler()

y_scaled = s_scaler.fit_transform(y.values.reshape(-1,1))
X_scaled = s_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


In [103]:

import tensorflow as ts
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout


nodes_number = int((X.shape[1] + 1) * 2/3)

model = Sequential()
model.add(Input(shape= (X.shape[1],)))
model.add(Dense(nodes_number, activation='relu'))
model.add(Dense(nodes_number, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.summary()



Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_79 (Dense)                │ (None, 6)              │            54 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_80 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 103 (412.00 B)

 Trainable params: 103 (412.00 B)

 Non-trainable params: 0 (0.00 B)

Install cuda toolkit for GPU on desktop

Tensorflow GPU version (maybe try conda env)

In [104]:
model.compile(optimizer="adam",
              loss="mse",
              metrics=['mae'])

model.fit(X_train, y_train, epochs=200, verbose=2)

model.summary()

Epoch 1/200
818/818 - 2s - 2ms/step - loss: 0.9977 - mae: 0.6239
Epoch 2/200
818/818 - 1s - 980us/step - loss: 0.9357 - mae: 0.5891
Epoch 3/200
818/818 - 1s - 983us/step - loss: 0.9083 - mae: 0.5717
Epoch 4/200
818/818 - 1s - 981us/step - loss: 0.8984 - mae: 0.5686
Epoch 5/200
818/818 - 1s - 961us/step - loss: 0.8894 - mae: 0.5635
Epoch 6/200
818/818 - 1s - 924us/step - loss: 0.8882 - mae: 0.5623
Epoch 7/200
818/818 - 1s - 921us/step - loss: 0.8842 - mae: 0.5607
Epoch 8/200
818/818 - 1s - 914us/step - loss: 0.8808 - mae: 0.5594
Epoch 9/200
818/818 - 1s - 912us/step - loss: 0.8817 - mae: 0.5581
Epoch 10/200
818/818 - 1s - 920us/step - loss: 0.8789 - mae: 0.5571
Epoch 11/200
818/818 - 1s - 916us/step - loss: 0.8762 - mae: 0.5578
Epoch 12/200
818/818 - 1s - 1ms/step - loss: 0.8749 - mae: 0.5561
Epoch 13/200
818/818 - 1s - 1ms/step - loss: 0.8755 - mae: 0.5555
Epoch 14/200
818/818 - 1s - 1ms/step - loss: 0.8720 - mae: 0.5532
Epoch 15/200
818/818 - 1s - 920us/step - loss: 0.8690 - mae: 0.55

KeyboardInterrupt: 

In [101]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = model.predict(X_test)

score = r2_score(y_test, y_pred)

print(score)
print(mean_squared_error(y_test, y_pred))

205/205 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step
0.16040197476139872
0.8206046324009719


it takes absurd amount of time to make a very poor predict prediction, probably need to preprocess data again

Best score with such settings

nodes_number = int((X.shape[1] + 1) * 2/3)

model = Sequential()

model.add(Input(shape= (X.shape[1],)))

model.add(Dense(nodes_number, activation='relu'))

model.add(Dense(nodes_number, activation='relu'))

model.add(Dense(1))


with standard scaler data applied to both X and y

0.21329008825613271

0.7689129542072395

Probably it worth to try use 62 features with location and timestamps excluded. Could possibly apply PCA


In [108]:
from keras import Sequential
from sentry_sdk.utils import epoch

# -*- coding: utf-8 -*-
"""
Created on Wed Oct 29 15:40:04 2025

@author: zemsk


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
import os


path = "./coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral.csv")

df = pd.read_csv(filename_read)

# # Removed only locations and labels, no real data touched
# df.drop("Ocean_Name", axis=1, inplace=True)
# df.drop("Country_Name", axis=1, inplace=True)
# df.drop("Sample_ID", axis=1, inplace=True)
# df.drop("Date_Year", axis=1, inplace=True)
# df.drop("Bleaching_Level", axis=1, inplace=True)
# df.drop("Realm_Name", axis=1, inplace=True)
#
# cols_to_drop = [
#     'Site_ID', 'Data_Source', 'Latitude_Degrees', 'Longitude_Degrees',
#     'Reef_ID', 'Ecoregion_Name', 'State_Island_Province_Name',
#     'City_Town_Name', 'Site_Name', 'Date', 'Date_Day', 'Date_Month',
#     'Site_Comments', 'Sample_Comments', 'Bleaching_Comments'
# ]
#
# df.drop(cols_to_drop, axis=1, inplace=True)
# #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # For the sake of bigger dataset this feature is dropped
# df.drop("Percent_Cover", axis=1, inplace=True)
# df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
# df.drop("Temperature_Maximum", inplace=True, axis=1)
#
# label_encoder = LabelEncoder()
# df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
#df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
# df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df.dropna(inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41361 entries, 0 to 41360
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Site_ID                                41361 non-null  int64  
 1   Sample_ID                              41361 non-null  int64  
 2   Data_Source                            41361 non-null  object 
 3   Latitude_Degrees                       41361 non-null  float64
 4   Longitude_Degrees                      41361 non-null  float64
 5   Ocean_Name                             41361 non-null  object 
 6   Reef_ID                                41361 non-null  object 
 7   Realm_Name                             41361 non-null  object 
 8   Ecoregion_Name                         41361 non-null  object 
 9   Country_Name                           41361 non-null  object 
 10  State_Island_Province_Name             41361 non-null  object 
 11  Ci

C:\Users\zemsk\AppData\Local\Temp\ipykernel_11792\3630821099.py:27: DtypeWarning: Columns (13,15,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename_read)
